# Домашнее задание 3. Парсинг, Git и тестирование на Python

**Цели задания:**

* Освоить базовые подходы к web-scraping с библиотеками `requests` и `BeautisulSoup`: навигация по страницам, извлечение HTML-элементов, парсинг.
* Научиться автоматизировать задачи с использованием библиотеки `schedule`.
* Попрактиковаться в использовании Git и оформлении проектов на GitHub.
* Написать и запустить простые юнит-тесты с использованием `pytest`.


В этом домашнем задании вы разработаете систему для автоматического сбора данных о книгах с сайта [Books to Scrape](http://books.toscrape.com). Нужно реализовать функции для парсинга всех страниц сайта, извлечения информации о книгах, автоматического ежедневного запуска задачи и сохранения результата.

Важной частью задания станет оформление проекта: вы создадите репозиторий на GitHub, оформите `README.md`, добавите артефакты (код, данные, отчеты) и напишете базовые тесты на `pytest`.



In [ ]:
! pip install -q schedule pytest # установка библиотек, если ещё не

In [15]:
# Библиотеки, которые могут вам понадобиться
# При необходимости расширяйте список
import time
import requests
import schedule
from bs4 import BeautifulSoup
from bs4.element import Tag
import re
import cProfile
from tqdm import tqdm
import threading
import json
import os

## Задание 1. Сбор данных об одной книге (20 баллов)

В этом задании мы начнем подготовку скрипта для парсинга информации о книгах со страниц каталога сайта [Books to Scrape](https://books.toscrape.com/).

Для начала реализуйте функцию `get_book_data`, которая будет получать данные о книге с одной страницы (например, с [этой](http://books.toscrape.com/catalogue/a-light-in-the-attic_1000/index.html)). Соберите всю информацию, включая название, цену, рейтинг, количество в наличии, описание и дополнительные характеристики из таблицы Product Information. Результат достаточно вернуть в виде словаря.

**Не забывайте про соблюдение PEP-8** — помимо качественно написанного кода важно также документировать функции по стандарту:
* кратко описать, что она делает и для чего нужна;
* какие входные аргументы принимает, какого они типа и что означают по смыслу;
* аналогично описать возвращаемые значения.

*P. S. Состав, количество аргументов функции и тип возвращаемого значения можете менять как вам удобно. То, что написано ниже в шаблоне — лишь пример.*

In [16]:

_number_pattern    = re.compile('[\d.]+') # Паттерн поиска числа
_rating_pattern    = re.compile('star-rating .*') # Паттерн поиска строкового значения рейтинги(One, Two, etc.)
_main_data_pattern = re.compile('.* product_main') # Паттерн поиска имени класса основных данных книги

def get_tag(context: Tag, tag_name: str, class_name: str|re.Pattern = '') -> Tag|None:
   """
   Находит объект Tag с указанным именем и классом в переданном контексте.
    
    Args:
        context (Tag): Объект bs4.element.Tag, в котором осуществляется поиск
        tag_name (str): Название HTML-тега для поиска (например, 'div', 'span', 'a')
        class_name (str|re.Pattern, optional): Название класса или регулярное выражение 
                                              для поиска по классу. По умолчанию ''.
    
    Returns:
        Tag|None: Первый найденный тег, соответствующий критериям, или None если тег не найден
                  или найденный объект не является тегом.
    """
  
   tag = context.find(tag_name, class_=class_name) if class_name else context.find(tag_name, class_=None)
   return tag if type(tag) == Tag else None



def _set_dict_value(data: dict[str, str|int|float], key: str, element: Tag|None, data_type: str) -> None:
   """
   Извлекает данные из Tag-элемента и добавляет их в словарь.
    
   Args:
      data (dict[str, str|int|float]): Словарь, в который добавляется значение
      key (str): Ключ для записи значения в словарь
      element (Tag|None): HTML-элемент BeautifulSoup Tag или None
      data_type (str): Тип извлекаемых данных: 'text', 'number' или 'rating'
    
   Returns:
      None: Функция модифицирует переданный словарь data
    
   Notes:
      - Для data_type 'text': извлекает текстовое содержимое элемента
      - Для data_type 'number': ищет числовое значение в тексте с помощью регулярного выражения
      - Для data_type 'rating': определяет рейтинг по второму слову в названии класса (0-5 звезд)
      - Если передан неверный аргумент element, значение в словаре не меняется
    """
   if not element:
      return
   elif data_type not in ('text', 'number', 'rating'):
      return
   elif data_type == 'text':
      data[key] = element.text.strip()
   elif data_type == 'number':
      match = re.search(_number_pattern, element.text)
      if match:
         data[key] = float(match.group()) if '.' in match.group() else int(match.group())
   elif data_type == 'rating':
      stars_count = ('zero', 'one', 'two', 'three', 'four', 'five')
      rating = element['class'][1].lower()
      if rating in stars_count:
         data[key] = stars_count.index(rating)

def get_rows(parent: Tag) -> dict[str, str]:
   """
    Извлекает данные из таблицы и возвращает в виде словаря.
    
    Args:
        parent: Родительский Tag-элемент, содержащий таблицу
    
    Returns:
        dict: Словарь, где ключи - тексты из ячеек <th>, 
              значения - тексты из соответствующих ячеек <td>
    """
   rows = {}
   for row in parent.find_all('tr'):
      if type(row) != Tag:
         continue
      key   = get_tag(row, 'th')
      value = get_tag(row, 'td')
      if key:
         _set_dict_value(rows, key.text, value, 'text')
         
   return rows

def get_book_data(book_url: str) -> dict[str, str|int|float|dict[str, str]]:
   """
    Извлекает данные о книге с веб-страницы.
    
    Args:
        book_url (str): URL-адрес страницы книги для парсинга
    
    Returns:
        dict: Словарь с данными о книге, содержащий:
            - title (str): Название книги
            - price (int|float): Цена книги
            - rating (int): Рейтинг от 0 до 5
            - available (int): Количество доступных экземпляров
            - description (str): Описание книги
            - additional_info (dict): Дополнительная информация из таблицы
    """

   # НАЧАЛО ВАШЕГО РЕШЕНИЯ
   data = {
      'title': '',
      'price': 0, 
      'rating': 0, 
      'available': 0, 
      'description': '', 
      'additional_info': {}
      }

   response = requests.get(book_url) 
   response.encoding = 'utf-8'

   # Выбрасывает исключение, если запрос неудачный (код начинается не с 2*)  
   response.raise_for_status()

   soup = BeautifulSoup(response.text, 'html.parser')

   data_root = get_tag(soup, 'article', 'product_page')

   if not data_root:
      return data

   main_data = get_tag(data_root , 'div', _main_data_pattern)

   if not main_data:
      return data

   title_elem        = get_tag(main_data, 'h1')
   description_elem  = get_tag(data_root, 'p')
   rating_elem       = get_tag(main_data, 'p', _rating_pattern)
   price_elem        = get_tag(main_data, 'p', 'price_color')
   available_elem    = get_tag(main_data, 'p', 'instock availability')

   _set_dict_value(data, 'title'       , title_elem      , 'text')
   _set_dict_value(data, 'description' , description_elem, 'text')
   _set_dict_value(data, 'price'       , price_elem      , 'number')
   _set_dict_value(data, 'available'   , available_elem  , 'number')
   _set_dict_value(data, 'rating'      , rating_elem     , 'rating')

   table = get_tag(data_root, 'table', 'table table-striped')

   if table:
      data['additional_info'] = get_rows(table)

   return data
   # КОНЕЦ ВАШЕГО РЕШЕНИЯ


In [17]:

# Используйте для самопроверки
book_url = 'http://books.toscrape.com/catalogue/a-light-in-the-attic_1000/index.html'
#book_url = 'https://books.toscrape.com/catalogue/the-bridge-to-consciousness-im-writing-the-bridge-between-science-and-our-old-and-new-beliefs_840/index.html'
#cProfile.run('get_book_data(book_url)')
get_book_data(book_url)

{'title': 'A Light in the Attic',
 'price': 51.77,
 'rating': 3,
 'available': 22,
 'description': "It's hard to imagine a world without A Light in the Attic. This now-classic collection of poetry and drawings from Shel Silverstein celebrates its 20th anniversary with this special edition. Silverstein's humorous and creative verse can amuse the dowdiest of readers. Lemon-faced adults and fidgety kids sit still and read these rhythmic words and laugh and smile and love th It's hard to imagine a world without A Light in the Attic. This now-classic collection of poetry and drawings from Shel Silverstein celebrates its 20th anniversary with this special edition. Silverstein's humorous and creative verse can amuse the dowdiest of readers. Lemon-faced adults and fidgety kids sit still and read these rhythmic words and laugh and smile and love that Silverstein. Need proof of his genius? RockabyeRockabye baby, in the treetopDon't you know a treetopIs no safe place to rock?And who put you up th

## Задание 2. Сбор данных обо всех книгах (20 баллов)

Создайте функцию `scrape_books`, которая будет проходиться по всем страницам из каталога (вида `http://books.toscrape.com/catalogue/page-{N}.html`) и осуществлять парсинг всех страниц в цикле, используя ранее написанную `get_book_data`.

Добавьте аргумент-флаг, который будет отвечать за сохранение результата в файл: если он будет равен `True`, то информация сохранится в ту же папку в файл `books_data.txt`; иначе шаг сохранения будет пропущен.

**Также не забывайте про соблюдение PEP-8**

In [2]:
# Корневая ссылка для парсинга и скрапинга книг
_root_url = 'https://books.toscrape.com/catalogue/'

def _get_page_soup(page_number: int) -> BeautifulSoup:
    """
    Получает HTML-страницу каталога и возвращает объект BeautifulSoup.
    
    Args:
        page_number (int): Номер страницы каталога для загрузки
    
    Returns:
        BeautifulSoup: Объект для парсинга HTML-страницы
    
    Raises:
        requests.HTTPError: Если HTTP-запрос завершился с ошибкой
    """
    response = requests.get(f'{_root_url}page-{page_number}.html')
    # Выбрасывает исключение, если запрос неудачный (код начинается не с 2*)
    response.raise_for_status()
    return BeautifulSoup(response.text, 'html.parser')

def _get_pages_count() -> int:
    """
    Определяет количество страниц в каталоге.
    
    Returns:
        int: Общее количество страниц или 0 при ошибке
    """
    try:
        soup = _get_page_soup(1)
    except:
        return 0
    
    counter_elem = get_tag(soup, 'li', 'current')
    
    if not counter_elem:
        return 0
    
    pattern = re.compile('of (\d+)')
    match = re.search(pattern, counter_elem.text)

    return int(match.group(1)) if match else 0


def _parse_page(page_number: int,  books_data: list[dict], pbar: tqdm|None = None) -> None:
    """
    Парсит страницу каталога и добавляет данные книг в список.
    Функция используется для работы в многопоточном режиме.
    
    Args:
        page_number (int): Номер страницы каталога для парсинга
        books_data (list[dict]): Список для добавления данных о книгах
        pbar (tqdm): Прогресс-бар для обновления статуса выполнения
    """
    try: 
        soup = _get_page_soup(page_number)
    except:
        return

    books_container = get_tag(soup, 'ol', 'row')

    if not books_container:
        return

    for heading in books_container.find_all('h3'):
        
        if type(heading) != Tag:
            continue
        
        link = get_tag(heading, 'a')
       
        if not link:
            continue
        
        href = link['href'] 

        if not href:
            continue

        url = _root_url + str(href)

        books_data.append(get_book_data(url))

    if pbar:
        pbar.update(1)


def scrape_books(is_save=False) -> list[dict]:
    """
    Собирает данные о всех книгах из каталога с использованием многопоточности.
    
    Args:
        is_save (bool, optional): Сохранять ли данные в файл. По умолчанию False.
    
    Returns:
        list[dict]: Список словарей с данными о книгах
    
    Process:
        - Определяет общее количество страниц в каталоге
        - Обрабатывает страницы параллельно в нескольких потоках
        - Сохраняет данные в файл при необходимости
        - Отображает прогресс выполнения через tqdm
    """
    # НАЧАЛО ВАШЕГО РЕШЕНИЯ
    
    # Количество потоков. В одном потоке обрабатывается одна страница
    threads_number = 25

    # Список, в который будут добавляться распрасенные данные книг
    books_data = []

    # Количество страниц
    pages_count = _get_pages_count()
    
    with tqdm(total=pages_count, desc='scraping pages') as pbar:

        for page_number in range(1, pages_count + 1, threads_number):
            
            threads = []

            for shift in range(threads_number):
                if page_number + shift > pages_count:
                    break
                threads.append(
                    threading.Thread(
                        target=_parse_page,
                        args=(page_number + shift, books_data, pbar))
                    ) 

            for thread in threads:
                thread.start()

            for thread in threads:
                thread.join()

    if is_save:
        with open('books_data.txt', 'w', encoding='utf-8') as f:
            f.write(json.dumps(books_data, indent=4))

    return books_data
    # КОНЕЦ ВАШЕГО РЕШЕНИЯ

NameError: name 'BeautifulSoup' is not defined

In [1]:
# Проверка работоспособности функции
res = scrape_books(is_save=True) # Допишите ваши аргументы
print(type(res), len(res)) # и проверки

# Проверим количество уникальных названий книг
titles = set(d['title'] for d in res)
print(len(titles)) # Результат = 999 названий

# Убедимся, что есть книги с одинаковыми названиями
l = []
for d in res:
    if l.count(d['title']):
        print(d['title'])
    l.append(d['title'])

NameError: name 'scrape_books' is not defined

## Задание 3. Настройка регулярной выгрузки (10 баллов)

Настройте автоматический запуск функции сбора данных каждый день в 19:00.
Для автоматизации используйте библиотеку `schedule`. Функция должна запускаться в указанное время и сохранять обновленные данные в текстовый файл.



Бесконечный цикл должен обеспечивать постоянное ожидание времени для запуска задачи и выполнять ее по расписанию. Однако чтобы не перегружать систему, стоит подумать о том, чтобы выполнять проверку нужного времени не постоянно, а раз в какой-то промежуток. В этом вам может помочь `time.sleep(...)`.

Проверьте работоспособность кода локально на любом времени чч:мм.



In [ ]:
# НАЧАЛО ВАШЕГО РЕШЕНИЯ

def run_autoscreping(start_time = '19:00', sleeping_time = 60) -> None:
    """
    Запускает автоматический парсинг книг по расписанию.
    
    Args:
        start_time (str): Время запуска парсинга в формате 'HH:MM'. По умолчанию '19:00'
        sleeping_time (int): Интервал проверки расписания в секундах. По умолчанию 60 секунд
    """
    schedule.every().day.at(start_time).do(scrape_books, True)
    while True:
        schedule.run_pending()
        time.sleep(sleeping_time)


# Запустите код, как он есть, для установки расписания согласно заданию
is_test = False

# Раскомментируйте строку ниже, если хотите выполнить тест
#is_test = True 

if is_test:
    # Код установит сбор данных сбор данных через минуту от текущего времени
    current_time_plus_a_minute = time.strftime('%H:%M', time.localtime(time.time() + 60))
    run_autoscreping(current_time_plus_a_minute)
else:
    run_autoscreping()

# КОНЕЦ ВАШЕГО РЕШЕНИЯ

## Задание 4. Написание автотестов (15 баллов)

Создайте минимум три автотеста для ключевых функций парсинга — например, `get_book_data` и `scrape_books`. Идеи проверок (можете использовать свои):

* данные о книге возвращаются в виде словаря с нужными ключами;
* список ссылок или количество собранных книг соответствует ожиданиям;
* значения отдельных полей (например, `title`) корректны.

Оформите тесты в отдельном скрипте `tests/test_scraper.py`, используйте библиотеку `pytest`. Убедитесь, что тесты проходят успешно при запуске из терминала командой `pytest`.

Также выведите результат их выполнения в ячейке ниже.

**Не забывайте про соблюдение PEP-8**


In [14]:
# Ячейка для демонстрации работоспособности
# Сам код напишите в отдельном скрипте

! pytest ../tests/test_scraper.py -v

============================= test session starts =============================
platform win32 -- Python 3.10.6, pytest-8.4.2, pluggy-1.6.0 -- C:\Users\ardd\AppData\Local\Programs\Python\Python310\python.exe
cachedir: .pytest_cache
rootdir: d:\python\mipt\books_scraper
plugins: anyio-4.10.0, mock-3.15.1
collecting ... collected 5 items

..\tests\test_scraper.py::test_get_book_data PASSED                      [ 20%]
..\tests\test_scraper.py::test_scrape_books PASSED                       [ 40%]
..\tests\test_scraper.py::test_scrape_books_with_save_file PASSED        [ 60%]
..\tests\test_scraper.py::test_check_file_data PASSED                    [ 80%]
..\tests\test_scraper.py::test_remove_test_data PASSED                   [100%]

============================= 5 passed in 51.12s ==============================


## Задание 5. Оформление проекта на GitHub и работа с Git (35 баллов)

В этом задании нужно воспользоваться системой контроля версий Git и платформой GitHub для хранения и управления своим проектом. **Ссылку на свой репозиторий пришлите в форме для сдачи ответа.**

### Пошаговая инструкция и задания

**1. Установите Git на свой компьютер.**

* Для Windows: [скачайте установщик](https://git-scm.com/downloads) и выполните установку.
* Для macOS:

  ```
  brew install git
  ```
* Для Linux:

  ```
  sudo apt update
  sudo apt install git
  ```

**2. Настройте имя пользователя и email.**

Это нужно для подписи ваших коммитов, сделайте в терминале через `git config ...`.

**3. Создайте аккаунт на GitHub**, если у вас его еще нет:
[https://github.com](https://github.com)

**4. Создайте новый репозиторий на GitHub:**

* Найдите кнопку **New repository**.
* Укажите название, краткое описание, выберите тип **Public** (чтобы мы могли проверить ДЗ).
* Не ставьте галочку Initialize this repository with a README.

**5. Создайте локальную папку с проектом.** Можно в терминале, можно через UI, это не имеет значения.

**6. Инициализируйте Git в этой папке.** Здесь уже придется воспользоваться некоторой командой в терминале.

**7. Привяжите локальный репозиторий к удаленному на GitHub.**

**8. Создайте ветку разработки.** По умолчанию вы будете находиться в ветке `main`, создайте и переключитесь на ветку `hw-books-parser`.

**9. Добавьте в проект следующие файлы и папки:**

* `scraper.py` — ваш основной скрипт для сбора данных.
* `README.md` — файл с кратким описанием проекта:

  * цель;
  * инструкции по запуску;
  * список используемых библиотек.
* `requirements.txt` — файл со списком зависимостей, необходимых для проекта (не присылайте все из глобального окружения, создайте изолированную виртуальную среду, добавьте в нее все нужное для проекта и получите список библиотек через `pip freeze`).
* `artifacts/` — папка с результатами парсинга (`books_data.txt` — полностью или его часть, если весь не поместится на GitHub).
* `notebooks/` — папка с заполненным ноутбуком `HW_03_python_ds_2025.ipynb` и запущенными ячейками с выводами на экран.
* `tests/` — папка с тестами на `pytest`, оформите их в формате скрипта(-ов) с расширением `.py`.
* `.gitignore` — стандартный файл, который позволит исключить временные файлы при добавлении в отслеживаемые (например, `__pycache__/`, `.DS_Store`, `*.pyc`, `venv/` и др.).


**10. Сделайте коммит.**

**11. Отправьте свою ветку на GitHub.**

**12. Создайте Pull Request:**

* Перейдите в репозиторий на GitHub.
* Нажмите кнопку **Compare & pull request**.
* Укажите, что было добавлено, и нажмите **Create pull request**.

**13. Выполните слияние Pull Request:**

* Убедитесь, что нет конфликтов.
* Нажмите **Merge pull request**, затем **Confirm merge**.

**14. Скачайте изменения из основной ветки локально.**



### Требования к итоговому репозиторию

* Файл `scraper.py` с рабочим кодом парсера.
* `README.md` с описанием проекта и инструкцией по запуску.
* Папка `artifacts/` с результатом сбора данных (`.txt` файл).
* Папка `tests/` с тестами на `pytest`.
* Папка `notebooks/` с заполненным ноутбуком `HW_03_python_ds_2025.ipynb`.
* Pull Request с комментарием из ветки `hw-books-parser` в ветку `main`.
* Примерная структура:

  ```
  books_scraper/
  ├── artifacts/
  │   └── books_data.txt
  ├── notebooks/
  │   └── HW_03_python_ds_2025.ipynb
  ├── scraper.py
  ├── README.md
  ├── tests/
  │   └── test_scraper.py
  ├── .gitignore
  └── requirements.txt
  ```